In [184]:
import dlib
import cv2
import numpy as np
import math
import pandas as pd

In [185]:
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
detector = dlib.get_frontal_face_detector()

In [186]:
import numpy as np

def calculate_eye_aspect_ratio(landmarks):
    # Convert the dlib landmarks to a list of (x, y) tuples
    landmarks_list = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)]
    
    # Calculate the distances between specific points on the eye
    numerator1 = (np.linalg.norm(np.abs(np.array(landmarks_list[41]) - np.array(landmarks_list[37])))+
                 np.linalg.norm(np.abs(np.array(landmarks_list[40]) - np.array(landmarks_list[38]))))
    
    denominator1 = 2 * np.linalg.norm(np.abs(np.array(landmarks_list[39]) - np.array(landmarks_list[36])))
    
    # Calculate the eye aspect ratio
    eye_aspect_ratio1 = numerator1 / denominator1
    numerator2 = (np.linalg.norm(np.abs(np.array(landmarks_list[47]) - np.array(landmarks_list[43])))+
                 np.linalg.norm(np.abs(np.array(landmarks_list[46]) - np.array(landmarks_list[44]))))
    
    denominator2 = 2 * np.linalg.norm(np.abs(np.array(landmarks_list[45]) - np.array(landmarks_list[42])))
    
    eye_aspect_ratio2 = eye_aspect_ratio1 = numerator2 / denominator2
    eye_aspect_ratio = (eye_aspect_ratio1 + eye_aspect_ratio2)/2
    return eye_aspect_ratio


In [187]:
from scipy.spatial import distance
def calculate_combined_eye_distance(landmarks, left_eye_indices,right_eye_indices, anchor_index):
    landmarks = [(p.x, p.y) for p in landmarks.parts()]
    left_eye_landmarks = [landmarks[i] for i in left_eye_indices]
    right_eye_landmarks = [landmarks[i] for i in right_eye_indices]
    anchor_landmark = landmarks[anchor_index]
    combined_left_eye_distance = sum(abs(distance.euclidean(anchor_landmark, point)) for point in left_eye_landmarks)
    combined_right_eye_distance = sum(abs(distance.euclidean(anchor_landmark, point)) for point in right_eye_landmarks)
    combined_eye_distance = (combined_left_eye_distance + combined_right_eye_distance)/2

    return combined_eye_distance

In [188]:
def calculate_combined_eyebrow_distance(landmarks, right_eyebrow_indices, left_eyebrow_indices, anchor_index):
    landmarks = [(p.x, p.y) for p in landmarks.parts()]
    left_eyebrow_landmarks = [landmarks[i] for i in left_eyebrow_indices]
    right_eyebrow_landmarks = [landmarks[i] for i in right_eyebrow_indices]
    anchor_landmark = landmarks[anchor_index]
    combined_left_eyebrow_distance = sum(abs(distance.euclidean(anchor_landmark, point)) for point in left_eyebrow_landmarks)
    combined_right_eyebrow_distance = sum(abs(distance.euclidean(anchor_landmark, point)) for point in right_eyebrow_landmarks)
    combined_eyebrow_distance = (combined_right_eyebrow_distance + combined_left_eyebrow_distance)/2

    return combined_eyebrow_distance

In [198]:
import math
from scipy.spatial import distance

def calculate_mouth_aspect_ratio(landmarks, upper_lip_indices, lower_lip_indices, corner_index1, corner_index2):
    landmarks = [(p.x, p.y) for p in landmarks.parts()]

    upper_lip_landmarks = [landmarks[i] for i in upper_lip_indices]
    lower_lip_landmarks = [landmarks[j] for j in lower_lip_indices]
    corner_index54 = landmarks[corner_index1]
    corner_index48 = landmarks[corner_index2]

    numerator = sum(abs(distance.euclidean(np.array(i), np.array(j))) for i, j in zip(upper_lip_landmarks, lower_lip_landmarks))
    denominator = abs(distance.euclidean(corner_index54, corner_index48))

    # Calculate the mouth aspect ratio
    mouth_aspect_ratio = numerator / (3 * denominator)
    return mouth_aspect_ratio


In [190]:
from scipy.spatial import distance
def calculate_nose_to_jaw_ratio(landmarks,anchor_index,jaw_index,reference_index):
    landmarks = [(p.x, p.y) for p in landmarks.parts()]
    # Calculate the distances between specific points on the nose and jaw
    #anchor_index = 31  # Index of the tip of the nose
    #jaw_index = 3
    #numerator = np.linalg.norm(np.abs(np.array(landmarks[anchor_index]) - np.array(landmarks[jaw_index])))
    #reference_index = 15
    anchor_point = landmarks[anchor_index]
    jaw_point = landmarks[jaw_index]
    reference_point = landmarks[reference_index]
    numerator = abs(distance.euclidean(anchor_point, jaw_point))
    #denominator = np.linalg.norm(np.abs(np.array(landmarks[reference_index]) - np.array(landmarks[jaw_index])))
    denominator = abs(distance.euclidean(reference_point, jaw_point))
    # Calculate the nose-to-jaw ratio
    nose_to_jaw_ratio = numerator / denominator
    return nose_to_jaw_ratio

In [191]:
def calculate_nose_to_chin_ratio(landmarks,anchor_index,chin_index1,chin_index2,chin_index3):
    # Calculate the distances between specific points on the nose and chin
    landmarks = [(p.x, p.y) for p in landmarks.parts()]
    anchor_point = landmarks[anchor_index] # Index of the tip of the nose
    chin_point1 = landmarks[chin_index1]    # Index of the first chin landmark
    chin_point2 = landmarks[chin_index2]   # Index of the second chin landmark
    chin_point3 = landmarks[chin_index3]
    #numerator = 2 * (np.linalg.norm(np.abs(np.array(landmarks[anchor_index]) - np.array(landmarks[chin_index1]))))
    numerator  = 2 * (abs(distance.euclidean(anchor_point, chin_point1)))
    denominator = denominator = (abs(distance.euclidean(chin_point2, chin_point3)) - abs(distance.euclidean(landmarks[22], landmarks[9])))
    # Calculate the nose-to-chin ratio
    nose_to_chin_ratio = numerator / denominator
    return nose_to_chin_ratio

In [192]:
def calculate_face_area(landmarks):
    landmarks = [(p.x, p.y) for p in landmarks.parts()]
    d_p1_p31 = np.linalg.norm(np.array(landmarks[0]) - np.array(landmarks[30]))
    d_p2_p31 = np.linalg.norm(np.array(landmarks[1]) - np.array(landmarks[30]))
    d_p1_p2 = np.linalg.norm(np.array(landmarks[0]) - np.array(landmarks[1]))
    S = (d_p1_p31 + d_p2_p31 + d_p1_p2) / 2.0
    area = 0.5*(S * ((S - d_p1_p31)**2 * ((S - d_p2_p31)**2)) * ((d_p1_p2)**2))
    return area

In [193]:
import numpy as np

def calculate_head_motion(landmarks, P_values):
    landmarks = [(p.x, p.y) for p in landmarks.parts()]
    
    differences = [np.linalg.norm(np.abs(np.array(landmarks[p]) - np.array(landmarks[30]))) for p in P_values]
    
    result = 0.5 * np.sum(differences)
    return result


In [194]:
def face_landmarks(images):
    feature_seq = []
    for image in images:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)
        features=[]
        for face in faces:

            #features=[]
                   # Predict facial landmarks
            landmarks = predictor(gray, face)

            # Process and visualize landmarks (optional)
            for point in landmarks.parts():
                  x, y = point.x, point.y
                  cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
            #cv2.imshow("Image with Landmarks", image)
            #cv2.waitKey(0)
            #calculate eye aspect ratio
            
            #left_eye_indices = [37, 38, 39, 40, 41, 42]  # Indices for left eye in the 68-point landmark model
            #right_eye_indices = [43, 44, 45, 46, 47, 48]     
            left_eye_aspect_ratio = calculate_eye_aspect_ratio(landmarks)
            #right_eye_aspect_ratio = calculate_eye_aspect_ratio(landmarks, right_eye_indices)
            eye_aspect_ratio = left_eye_aspect_ratio
            #calculate eye distance
            reference_point_index = 31
            left_eye_corner_indices = [36, 37, 38, 39, 40, 41]
            right_eye_corner_indices = [42, 43, 44, 45, 46, 47]
            anchor_index = 30
            eye_distance = calculate_combined_eye_distance(landmarks, left_eye_corner_indices,right_eye_corner_indices, anchor_index)
             
            #calculate eyebrow distance
            right_eyebrow_indices = [22, 23, 24, 25, 26]
            left_eyebrow_indices = [17, 18, 19, 20, 21]
            anchor_index = 30
            eyebrow_distance = calculate_combined_eyebrow_distance(landmarks, right_eyebrow_indices, left_eyebrow_indices, anchor_index)
            
            #calculate mouth aspect ratio
            upper_lip_indices = [61, 62, 63]
            lower_lip_indices = [67, 66, 65]
            corner_index1 = 54
            corner_index2 = 48
            mouth_aspect_ratio = calculate_mouth_aspect_ratio(landmarks,upper_lip_indices,lower_lip_indices,corner_index1,corner_index2)
            #nose to jaw ratio
            anchor_index = 30  # Index of the tip of the nose
            jaw_index = 2
            reference_index = 14
            nose_to_jaw_ratio = calculate_nose_to_jaw_ratio(landmarks,anchor_index, jaw_index,reference_index)
            #nose to chin ratio
            anchor_index = 30  # Index of the tip of the nose
            chin_index1 = 8    # Index of the first chin landmark
            chin_index2 = 21    # Index of the second chin landmark
            chin_index3 = 7
            nose_to_chin_ratio = calculate_nose_to_chin_ratio(landmarks,anchor_index,chin_index1,chin_index2,chin_index3)
            #calculate face area
            face_area = calculate_face_area(landmarks)
            #Calculate head motion
            P_values=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
            head_motion = calculate_head_motion(landmarks, P_values)
            print("eye_aspect_ratio:", eye_aspect_ratio)
            print("Combined Eye Distance for Both Eyes:", eye_distance)
            print("Combined Eyebrow Distance for Both Eyebrows:", eyebrow_distance)
            print("Mouth Aspect Ratio:", mouth_aspect_ratio)
            print("Nose-to-Jaw Ratio:", nose_to_jaw_ratio)
            print("Nose-to-Chin Ratio:", nose_to_chin_ratio)
            print("Face Area:", face_area)
            print("Head Motion:", head_motion)
            features.append(eye_aspect_ratio)
            features.append(eye_distance)
            features.append(eyebrow_distance)
            features.append(mouth_aspect_ratio)
            features.append(nose_to_jaw_ratio)
            features.append(nose_to_chin_ratio)
            features.append(face_area)
            features.append(head_motion)
            print(features)
            break;
        feature_seq.append(features)          
    return feature_seq
    #print(feature_seq)
             
cv2.destroyAllWindows()





In [195]:
import cv2
import time
import os


In [196]:
import os
import cv2
import pandas as pd

# Define the root folder containing subfolders and videos
root_folder = "C:\\Users\\acer\\Desktop\\Project\\Dataset"

# Define the folder for CSV files
csv_output_folder = "C:\\Users\\acer\\Desktop\\Project\\featureset"

# Define the interval in seconds for frame extraction
interval = 2  # 2 seconds

# Create the folder for CSV files if it doesn't exist
os.makedirs(csv_output_folder, exist_ok=True)

# Function to normalize a row based on its first value
def normalize_row(row):
    first_value = row.iloc[0]  # Use iloc to access values by position
    return row / first_value


# Iterate through subfolders within the root folder
for class_folder in os.listdir(root_folder):
    class_folder_path = os.path.join(root_folder, class_folder)
    
    # Check if the class folder is a directory
    if os.path.isdir(class_folder_path):
        print(f"Processing class folder: {class_folder}")
        
        # Create a folder to store CSV files for this class
        class_csv_folder = os.path.join(csv_output_folder, f"{class_folder}_csv")
        os.makedirs(class_csv_folder, exist_ok=True)
        
        # Iterate through video files in the class folder
        for video_file in os.listdir(class_folder_path):
            video_path = os.path.join(class_folder_path, video_file)
            print(f"Processing video: {video_path}")

            cap = cv2.VideoCapture(video_path)
            
            # Initialize variables
            frames = []
            frame_count = 0
            last_capture_time = 0
            all_feature_seq = []

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    print("No more frames")
                    break
                
                current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
                
                # Check if it's time to capture a frame
                if current_time - last_capture_time >= interval:
                    last_capture_time = current_time
                    
                       # Apply brightness adjustment
                       #brightness_factor = np.random.uniform(*brightness_factor_range)
                       #frame = cv2.convertScaleAbs(frame, alpha=brightness_factor, beta=0)

                    frames.append(frame)
                    frame_count += 1
                    frame_filename = f'frame_{frame_count:04d}.jpg'
                    cv2.imwrite(frame_filename, frame)
                    print(f"Captured frame {frame_count} at {current_time:.2f} seconds")
                    # Process frames and extract features using your 'face_landmarks' function
                    #feature_seq = face_landmarks(frames)
                    #all_feature_seq.extend(feature_seq)

            cap.release()
            cv2.destroyAllWindows()
            feature_seq = face_landmarks(frames)
            
            all_feature_seq.extend(feature_seq)
            # Create a Pandas DataFrame with the feature data
            column_names = ['Eye Aspect Ratio', 'Eye Distance', 'Eyebrow Distance', 'Mouth Aspect Ratio', 'Nose-to-Jaw Ratio','Nose-to-Chin Ratio','Face Area','Head Motion']
            df = pd.DataFrame(all_feature_seq, columns=column_names)
            df_normalized = df.apply(normalize_row, axis=1)
            # Create a CSV file for this video and save it in the class-specific folder
            video_name = os.path.splitext(video_file)[0]
            csv_file_path = os.path.join(class_csv_folder, f"{video_name}.csv")
            df_normalized.to_csv(csv_file_path, index=False)

            print(f"Normalized CSV file saved at: {csv_file_path}")
print(all_feature_seq)
print("Processing complete.")


Processing class folder: albhudam
Processing video: C:\Users\acer\Desktop\Project\Dataset\albhudam\AL1.mp4
Captured frame 1 at 2.00 seconds
Captured frame 2 at 4.00 seconds
Captured frame 3 at 6.00 seconds
Captured frame 4 at 8.00 seconds
Captured frame 5 at 10.00 seconds
No more frames
eye_aspect_ratio: 0.33262192186991835
Combined Eye Distance for Both Eyes: 692.9688515376037
Combined Eyebrow Distance for Both Eyebrows: 939.6774366366116
Mouth Aspect Ratio: 0.014024039375301976
Nose-to-Jaw Ratio: 0.6204099746017675
Nose-to-Chin Ratio: -151.61055564429194
Face Area: 103611474476.47478
Head Motion: 2515.767760223113
[0.33262192186991835, 692.9688515376037, 939.6774366366116, 0.014024039375301976, 0.6204099746017675, -151.61055564429194, 103611474476.47478, 2515.767760223113]
eye_aspect_ratio: 0.3239201057944745
Combined Eye Distance for Both Eyes: 688.8029680721668
Combined Eyebrow Distance for Both Eyebrows: 906.5173601339885
Mouth Aspect Ratio: 0.026863369199499706
Nose-to-Jaw Ratio:

C:\Users\acer\AppData\Local\Temp\ipykernel_3584\745919922.py:12: RuntimeWarning: divide by zero encountered in scalar divide
  nose_to_chin_ratio = numerator / denominator


eye_aspect_ratio: 0.24898558337823445
Combined Eye Distance for Both Eyes: 564.8006841979739
Combined Eyebrow Distance for Both Eyebrows: 637.0087118605511
Mouth Aspect Ratio: 0.03691670113307599
Nose-to-Jaw Ratio: 0.4466068231315184
Nose-to-Chin Ratio: inf
Face Area: 996649957.2494912
Head Motion: 1497.7065164394826
[0.24898558337823445, 564.8006841979739, 637.0087118605511, 0.03691670113307599, 0.4466068231315184, inf, 996649957.2494912, 1497.7065164394826]
eye_aspect_ratio: 0.18210287071733153
Combined Eye Distance for Both Eyes: 547.2530824954565
Combined Eyebrow Distance for Both Eyebrows: 638.4543312447659
Mouth Aspect Ratio: 0.15364734684683407
Nose-to-Jaw Ratio: 0.4667508204516049
Nose-to-Chin Ratio: 355.3663627611807
Face Area: 1403436185.2767422
Head Motion: 1521.423984639795
[0.18210287071733153, 547.2530824954565, 638.4543312447659, 0.15364734684683407, 0.4667508204516049, 355.3663627611807, 1403436185.2767422, 1521.423984639795]
eye_aspect_ratio: 0.21064353332528413
Combin